# Introduction to Visual Edit's CRUD Python API

[Visual Edit](https://dataiku.github.io/dss-visual-edit/) is a Dataiku plugin that helps build apps to validate and edit data within pipelines. As an alternative to using the plugin's Visual Webapp (no-code), you can build your own webapp with a custom front-end and use the plugin's CRUD API for data persistence.

This notebooks provides an overview of the API and a hands-on demo. We strongly recommend following the [Visual Webapp guide](https://dataiku.github.io/dss-visual-edit/get-started) first, to get an initial understanding of the plugin's features and concepts.

## Overview of API methods

The plugin's `DataEditor` Python class provides the following methods:

* Read data:
  * At row level: `get_row()`.
  * At dataset level: `get_original_df()`, `get_edited_df()`, `get_edited_cells_df()`, `get_editlog_df()`.
* Write data:
  * At row level: `create_row()`, `update_row()`, `delete_row()`.
  * At dataset level: `empty_editlog()`.

Find more information about each method in the [API reference documentation](https://dataiku.github.io/dss-visual-edit/backend/).


## Prerequisites

When running this code in Dataiku, a code environment must be specified.

* One was created upon installation of the plugin, but it can't be selected from the list of available code environments as it can only be used by the plugin's visual components (Visual Webapp and Recipes).
* You'll need to create a new code environment and to make sure it contains the same Python version and packages as the plugin's.
* These can be found from the Code Envs tab of your Dataiku instance's Administration panel at `/admin/code-envs/design/`).

## Demo

Import dataiku package and define data editing properties:

In [1]:
import dataiku

original_ds_name = "customers"
primary_keys = ["name"]
editable_column_names = ["address", "dob"]

### Initial setup

Create the original dataset:

In [2]:
client = dataiku.api_client()
project_key = dataiku.default_project_key()
project = client.get_project(project_key)
builder = project.new_managed_dataset(original_ds_name)
builder.with_store_into("filesystem_folders")
builder.create(overwrite=True)

Write an empty dataframe with the proper columns:

In [3]:
from pandas import DataFrame

original_ds = dataiku.Dataset(original_ds_name)
original_df = DataFrame(columns=primary_keys + editable_column_names)
original_ds.write_with_schema(original_df)

0 rows successfully written (Mkd9Htu5UI)


### Create an instance of `DataEditor`

Import from plugin:

In [4]:
dataiku.use_plugin_libs("visual-edit")
DataEditor = dataiku.import_from_plugin("visual-edit", "DataEditor")

Instantiate:

In [5]:
de = DataEditor.DataEditor(
    original_ds_name=original_ds_name,
    primary_keys=primary_keys,
    editable_column_names=editable_column_names,
)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ERROR:root:Failed to retrieve webapp url.
Traceback (most recent call last):
  File "/data/dataiku/dss_data/plugins/installed/visual-edit/python-lib/DataEditor.py", line 61, in __init_webapp_url__
    webapp_id = find_webapp_id(self.original_ds_name)
  File "/data/dataiku/dss_data/plugins/installed/visual-edit/python-lib/webapp_utils.py", line 40, in find_webapp_id
    return webapps_edit_df[
  File "/data/dataiku/dss_data/code-envs/python/python_39_visualedit/lib/python3.9/site-packages/pandas/core/indexing.py", line 879, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/data/dataiku/dss_data/code-envs/python/python_39_visualedit/lib/python3.9/site-packages/pandas/core/indexing.py", line 1496, i

0 rows successfully written (WNgJ9eqnWO)


This creates the editlog, edits, and edited datasets, and the "replay edits" and "apply edits" recipes in between, if they don't already exist. Those datasets are created on the same connection as the original dataset.

Quick check on the original and edited data:

In [6]:
de.get_original_df()

(Empty DataFrame
 Columns: [name, address, dob]
 Index: [],
 ['name'],
 [],
 ['address', 'dob'])

In [7]:
de.get_edited_df()

,name,address,dob


### Create

In [8]:
de.create_row(primary_keys={"name": "Louis Dorard"}, column_values={"address": "TBD"})

1 rows successfully written (YSipVW9AK3)


'Row successfully created'

In [9]:
de.create_row(
    primary_keys={"name": "New name"},
    column_values={
        "address": "New address",
    },
)

1 rows successfully written (ErRmkg3aKM)


'Row successfully created'

### Update

In [10]:
de.update_row(primary_keys={"name": "Louis Dorard"}, column="address", value="France")

1 rows successfully written (8L9DpRgCBQ)


### Delete

In [11]:
de.delete_row(primary_keys={"name": "New name"})

1 rows successfully written (8aNe84cPFp)


Note that this still outputs that a 'row' was successfully written: this is a row in the editlog!

### Read

Original data (should still be empty):

In [12]:
de.get_original_df()

(Empty DataFrame
 Columns: [name, address, dob]
 Index: [],
 ['name'],
 [],
 ['address', 'dob'])

Edited data (i.e. _all_ rows):

In [13]:
de.get_edited_df()

,name,address,dob
0,Louis Dorard,France,NaN


There's only one row: this makes sense, because we created 2 and deleted 1.

Edited cells (i.e. _edited_ rows only):

In [14]:
de.get_edited_cells_df()

,name,address,dob,last_edit_date,last_action,first_action
0,Louis Dorard,France,NaN,2024-05-22T14:25:26.133719+00:00,update,create
1,New name,New address,NaN,2024-05-22T14:25:27.668387+00:00,delete,create


This provides more information on how we "got here". There was another row (`name` = "New name") that got deleted (see value of `last_action` column)!

Let's see the full history of what happened:

In [ ]:
de.get_editlog_df()

### Reset (USE WITH CAUTION)

In [ ]:
de.empty_editlog()

## Using the API in a Dash webapp

See https://dataiku.github.io/dss-visual-edit/dash-examples